In [1]:
import tensorflow as tf
import os
import sys
import go_datafile_reader
import model
import numpy as np
import model_eval
reload(model)
reload(go_datafile_reader)
reload(model_eval)

<module 'model_eval' from 'model_eval.pyc'>

In [2]:
train_dir = "/home/justin/Programming/GoAI/MovePredictionCNN/data/input/train"
test_dir = "/home/justin/Programming/GoAI/MovePredictionCNN/data/input/test"
ckpt_path = "/home/justin/Programming/GoAI/MovePredictionCNN/data/working/board_eval_cnn_5layer.ckpt"

In [3]:
train_files = []
for subdir, dirs, files in os.walk(train_dir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".dat"):
            train_files.append(filepath)

test_files = []
for subdir, dirs, files in os.walk(test_dir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".dat"):
            test_files.append(filepath)

print "num train: %d, num test: %d" %(len(train_files), len(test_files))

num train: 14724, num test: 149


In [4]:
#note you may have to change the os limit for number of open files to use the RandomAccessFileReader
#you can do this with the command "sudo ulimit -n 20000"
#if sudo can't find the ulimit command try the following below
#sudo sh -c "ulimit -n 20000 && exec su $LOGNAME"

reader = go_datafile_reader.RandomAccessFileReader(train_files)
test_reader = go_datafile_reader.GoDatafileReader(test_files)

test_reader.num_epochs = 0
test_features = []
test_targets = []
while(test_reader.num_epochs == 0):
    final_state, _, feature_cube = test_reader.read_sample()
    test_features.append(feature_cube)
    test_targets.append(final_state)

Initialzed at pos: 4110
Initialzed at pos: 122067
Initialzed at pos: 47676
Initialzed at pos: 30003
Initialzed at pos: 36579
Initialzed at pos: 0
Initialzed at pos: 32058
Initialzed at pos: 25482
Initialzed at pos: 35757
Initialzed at pos: 27537
Initialzed at pos: 822
Initialzed at pos: 18084
Initialzed at pos: 88365
Initialzed at pos: 93708
Initialzed at pos: 83022
Initialzed at pos: 55485
Initialzed at pos: 66993
Initialzed at pos: 36990
Initialzed at pos: 82611
Initialzed at pos: 70692
Initialzed at pos: 4110
Initialzed at pos: 55074
Initialzed at pos: 411
Initialzed at pos: 76035
Initialzed at pos: 69459
Initialzed at pos: 24249
Initialzed at pos: 67404
Initialzed at pos: 23838
Initialzed at pos: 49731
Initialzed at pos: 53019
Initialzed at pos: 34935
Initialzed at pos: 5754
Initialzed at pos: 3288
Initialzed at pos: 65349
Initialzed at pos: 94941
Initialzed at pos: 50964
Initialzed at pos: 60828
Initialzed at pos: 73158
Initialzed at pos: 98229
Initialzed at pos: 64527
Initialzed 

In [5]:
print len(reader.open_files)

14724


In [6]:
x, ownership = model.place_holders()
y_conv = model.model(x)
loss = model.loss_function(ownership, y_conv)
train_op = model.train_step(loss)

prediction = tf.round(y_conv)
correct_prediction = tf.equal(ownership, prediction)
correct_count = tf.reduce_sum(tf.cast(correct_prediction, "float"))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver(tf.all_variables())
#saver.restore(sess, ckpt_path)

In [ ]:
training_accuracies = []
test_accuracies = []
for k in range(100000):
    x_batch, y_batch = reader.get_batch(50)
    _, loss_value, y_value = sess.run([train_op, loss, y_conv], feed_dict={x:x_batch, ownership:y_batch})
    if k % 10 == 0:
        acc = accuracy.eval(feed_dict={x:x_batch, ownership:y_batch})
        training_accuracies.append(acc)
        print k, loss_value, acc
            
    if k % 1000 == 0:
        test_accuracy = model_eval.test_accuracy(test_features, test_targets, x, ownership, correct_count)
        test_accuracies.append(test_accuracy)
        print "Test accuracy: %f" %test_accuracy
    if k % 1000 == 0:
        print "Saving Checkpoint..."
        saver.save(sess, ckpt_path)
    

8905.02 0.503712
Test accuracy: 0.503131
Saving Checkpoint...
7748.47 0.558504
6797.89 0.606537
6594.1 0.616122
5486.81 0.681773
5001.46 0.703989
5147.22 0.692687
4885.48 0.704377
4600.49 0.72277
4456.18 0.730138
4610.15 0.724875
4271.97 0.737064
4364.46 0.740222
4212.72 0.744321
4180.16 0.744986
4516.29 0.730416
4058.16 0.754792
4276.52 0.737064
4339.56 0.729529
3848.73 0.765928
4204.22 0.741939
3843.31 0.763546
3634.44 0.77651
3708.1 0.773629
4304.48 0.731967
4182.62 0.74205
3643.81 0.772687
3971.55 0.756011
3682.86 0.771524
3890.29 0.760554
3947.31 0.754571
3699.41 0.772355
3552.45 0.776343
3608.65 0.781219
3780.17 0.759446
3504.38 0.782881
4206.28 0.730249
3550.23 0.779778
3452.92 0.783047
3697.16 0.77313
3802.69 0.756343
3559.5 0.777119
3318.09 0.794626
3357.09 0.788643
3685.57 0.760443
3789.84 0.751911
3603.22 0.76903
3810.61 0.756787
3804.34 0.750803
4157.52 0.730416
3309.58 0.783546
3439.13 0.783712
3331.03 0.7841
3530.36 0.776787
3560.82 0.769252
3859.82 0.754737
3396.86 0.782

In [19]:
import matplotlib.pyplot as plt
import numpy as np

In [22]:
def rolling_mean(number_list, window = 20):
    means = np.zeros(len(number_list))
    for i in xrange(len(means)):
        sub_window = number_list[i-window+1:i+1]
        means[i] = np.mean(sub_window)
    return means

In [27]:
print len(training_accuracies)
means = rolling_mean(training_accuracies, 200)
plt.plot(training_accuracies, label="Batch Accuracy")
plt.plot(means, label= "Rolling Mean")
plt.title("Bad Data Mixing")
plt.xlabel("Batch Num / 10")
plt.ylabel("Accuracy")
plt.legend(loc=3)
plt.show()

2280


In [12]:
x_batch, y_batch = reader.get_batch(50)
y_val, y_preds = sess.run([y_conv, prediction], feed_dict={x:x_batch, ownership:y_batch})

In [13]:
idx = 0
y_pred = np.reshape(y_preds[idx], [19,19])
y_true = np.reshape(y_batch[idx], [19,19])
feature_cube = x_batch[idx]

model_eval.print_info(feature_cube, y_pred, y_true)

    

*******************   0000000000000000000   0000000110000111111
*******************   0000000000000000000   0000011100000111111
****0***********1**   0000000000000000000   0000000110000010111
**0****************   0000000000000000000   1001011110000000111
*******************   0000000000000000000   1111001111000001111
*******************   0000000000000000000   1111001110000000111
*******************   0000000000000000000   1111100100000000011
*******************   0000000000000000000   1100100100110010011
**1****************   0000000000000000000   0000111111111011111
****0**************   0000000000000000000   0000011111111111100
*******1***********   0000000000000000000   0000001100111111000
**00*1100**********   0000000000000000000   0000011000011111000
**011*0********0***   0000000000000000000   0001110000011110000
******0********10**   0000000000000000000   0001110000010111000
**0*1**0******010**   0000000000000000000   0000100000010111000
***010010**0*1*10**   000000000000000000